In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import json
with open("service1906_1506.json") as of:
    data = json.load(of)

In [3]:
computes = [c for c in data.keys() if c!="timespan"]
variables = [v for v in data[computes[0]] if v!='index' and v!='arrJob_scheduling']

In [4]:
#Check empty array
def getEmptyArr(data, c):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            print('c=', c)
            print('v=', v)
for c in computes:
    getEmptyArr(data, c)

c= compute-1-26
v= arrTemperature


In [5]:
def addTarget(cDf, predictedVar, predictedStep, target):
    cDf[target] = cDf[predictedVar].shift(-predictedStep)
    cDf.dropna(inplace=True)
    return cDf

In [6]:
def getComputeDf(data, c, predictedVar, predictedStep, target):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            return None
        else:
            for i in range(len(vArr[0])):
                cDf[v+str(i)] = vArr[:, i]
    cDf['timespan'] = pd.to_datetime(cDf['timespan'])
    return addTarget(cDf, predictedVar, predictedStep, target)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import FileLink
import codecs, json

In [8]:
def exportNPArrayToJson(a, fileName):
    b = a.tolist()
    json.dump(b, codecs.open(fileName, 'w', encoding='utf-8'))

In [9]:
def splitTrainTest(predictedVar):
    target = predictedVar + "_target"
    predictedSteps = 4
    df = pd.concat([x for x in [getComputeDf(data, c, predictedVar, predictedSteps, target) for c in computes] if type(x)!="NoneType"])
    
    df = df.reset_index().drop('index', axis=1)
    features = [x for x in df.columns if x not in ['compute', 'timespan', target]]
        
    X_dfs = []
    y = []
    numberOfSequences = 1
    sequenceSteps = 20
    # generate training data.
    for compute in computes:
        cDf = df[df['compute']==compute]
        if(len(cDf) > sequenceSteps):
            randSteps = np.random.randint(0, len(cDf)-sequenceSteps, numberOfSequences)
            for randStep in randSteps:
                X_dfs.append(cDf.iloc[randStep:randStep+sequenceSteps])
                y.append(X_dfs[-1][target].values[-1])

    X_train_dfs, X_test_dfs, y_train, y_test = train_test_split(X_dfs, y, test_size=0.33)
            
    #Scale data

    # combine the training data to create a scaler
    train_dfs = pd.concat(X_train_dfs)

    scaler = StandardScaler().fit(train_dfs[features].values)
    
    X_train = np.array([scaler.transform(item[features].values) for item in X_train_dfs])
    X_test = np.array([scaler.transform(item[features].values) for item in X_test_dfs])
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    exportNPArrayToJson(X_train, 'allData/'+target+'_X_train_HPCC_1_20.json')
    exportNPArrayToJson(X_test, 'allData/'+target+'_X_test_HPCC_1_20.json')
    exportNPArrayToJson(y_train, 'allData/'+target+'_y_train_HPCC_1_20.json')
    exportNPArrayToJson(y_test, 'allData/'+target+'_y_test_HPCC_1_20.json')
    
    FileLink('allData/'+target+'_X_train_HPCC_1_20.json')
    FileLink('allData/'+target+'_X_test_HPCC_1_20.json')
    FileLink('allData/'+target+'_y_train_HPCC_1_20.json')
    FileLink('allData/'+target+'_y_test_HPCC_1_20.json')
    
    return (X_train, X_test, y_train, y_test)

In [10]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten

# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=36, inter_op_parallelism_threads=36)))


def createModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, inputShape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=inputShape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do1 = Dropout(0.2)
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do2 = Dropout(0.2)
    
    flatten = Flatten()
    
    dense1 = Dense(d1Nodes, activation='relu')
    do3 = Dropout(0.2)
    
    dense2 = Dense(d2Nodes, activation='relu')
    do4 = Dropout(0.2)
    
    # output layer
    outL = Dense(1, activation='relu')
    # combine the layers
#     layers = [lstm1, do1, lstm2, do2, dense1, do3, dense2, do4, outL]
    layers = [lstm1, lstm2, flatten,  dense1, dense2, outL]
    # create the model
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='mse')
    return model

Using TensorFlow backend.


In [11]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
# ten fold
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True)
from keras.models import load_model
from datetime import datetime

msescores = []

allVariables = ['arrTemperature0', 'arrTemperature1', 'arrTemperature2', 'arrCPU_load0', 'arrMemory_usage0', 'arrFans_health0', 'arrFans_health1', 'arrFans_health2', 'arrFans_health3', 'arrPower_usage0']

start_time = datetime.now()
print(start_time)
for feature in allVariables:
    print(feature)
    X_train, X_test, y_train, Y_test = splitTrainTest(feature)

    counter= 0
    for trainIdx, testIdx in kfold.split(X_train, y_train):
        counter = counter + 1
        # create callbacks
        model_path = feature+'_best_model_fold'+str(counter)+"_"+datetime.now().strftime("%H_%M_%S")+'.h5'
        mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)
        # create model
        model = createModel(64, 64, 8, 8, (X_train.shape[1], X_train.shape[2]))
        model.fit(X_train[trainIdx], y_train[trainIdx], validation_data=(X_train[testIdx], y_train[testIdx]), batch_size=32, epochs=40, callbacks=[mc, es])
        # Done load the best model of this fold
        saved_model = load_model(model_path)
        msescores.append({'path': model_path, 'mse': saved_model.evaluate(X_train[testIdx], y_train[testIdx])})
        
end_time = datetime.now()

2019-10-24 13:43:43.146681
arrTemperature0
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 4ms/step - loss: 3808.0128 - val_loss: 3742.4236

Epoch 00001: val_loss improved from inf to 3742.42364, saving model to arrTemperature0_best_model_fold1_13_44_07.h5
Epoch 2/40
208/208 [==============================] - 0s 559us/step - loss: 3592.6900 - val_loss: 3278.8760

Epoch 00002: val_loss improved from 3742.42364 to 3278.87596, saving model to arrTemperature0_best_model_fold1_13_44_07.h5
Epoch 3/40
208/208 [==============================] - 0s 577us/step - loss: 2892.4404 - val_loss: 2277.7814

Epoch 00003: val_loss improved from 3278.87596 to 2277.78144, saving model to arrTemperature0_best_model_fold1_13_44_07.h5
Epoch 4/40
208/208 [==============================] - 0s 699us/step - loss: 1770.5178 - val_loss: 1193.4120

Epoch 00004: val_loss improved from 2277.78144 to 1193.41198, saving model to arrTemperature0_best_model_fold1_13_4

Epoch 15/40
208/208 [==============================] - 0s 546us/step - loss: 84.9030 - val_loss: 90.1653

Epoch 00015: val_loss improved from 91.44120 to 90.16532, saving model to arrTemperature0_best_model_fold2_13_44_14.h5
Epoch 16/40
208/208 [==============================] - 0s 558us/step - loss: 82.7951 - val_loss: 88.6183

Epoch 00016: val_loss improved from 90.16532 to 88.61833, saving model to arrTemperature0_best_model_fold2_13_44_14.h5
Epoch 17/40
208/208 [==============================] - 0s 553us/step - loss: 80.9148 - val_loss: 87.3944

Epoch 00017: val_loss improved from 88.61833 to 87.39437, saving model to arrTemperature0_best_model_fold2_13_44_14.h5
Epoch 18/40
208/208 [==============================] - 0s 562us/step - loss: 78.8113 - val_loss: 85.3234

Epoch 00018: val_loss improved from 87.39437 to 85.32336, saving model to arrTemperature0_best_model_fold2_13_44_14.h5
Epoch 19/40
208/208 [==============================] - 0s 554us/step - loss: 76.0952 - val_loss: 83.

208/208 [==============================] - 0s 536us/step - loss: 103.9297 - val_loss: 90.2168

Epoch 00020: val_loss did not improve from 80.13369
Epoch 21/40
208/208 [==============================] - 0s 544us/step - loss: 101.6474 - val_loss: 81.0950

Epoch 00021: val_loss did not improve from 80.13369
Epoch 00021: early stopping
104/104 [==============================] - 0s 1ms/step
arrTemperature1
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 4ms/step - loss: 2115.4535 - val_loss: 2360.3782

Epoch 00001: val_loss improved from inf to 2360.37816, saving model to arrTemperature1_best_model_fold1_13_44_52.h5
Epoch 2/40
208/208 [==============================] - 0s 569us/step - loss: 2113.7882 - val_loss: 2358.8300

Epoch 00002: val_loss improved from 2360.37816 to 2358.83004, saving model to arrTemperature1_best_model_fold1_13_44_52.h5
Epoch 3/40
208/208 [==============================] - 0s 585us/step - loss: 2112.3157 - val_lo

208/208 [==============================] - 0s 653us/step - loss: 2104.0169 - val_loss: 2349.9773

Epoch 00034: val_loss improved from 2349.97945 to 2349.97726, saving model to arrTemperature1_best_model_fold1_13_44_52.h5
Epoch 35/40
208/208 [==============================] - 0s 590us/step - loss: 2104.0150 - val_loss: 2349.9756

Epoch 00035: val_loss improved from 2349.97726 to 2349.97556, saving model to arrTemperature1_best_model_fold1_13_44_52.h5
Epoch 36/40
208/208 [==============================] - 0s 619us/step - loss: 2104.0136 - val_loss: 2349.9746

Epoch 00036: val_loss improved from 2349.97556 to 2349.97457, saving model to arrTemperature1_best_model_fold1_13_44_52.h5
Epoch 37/40
208/208 [==============================] - 0s 661us/step - loss: 2104.0126 - val_loss: 2349.9736

Epoch 00037: val_loss improved from 2349.97457 to 2349.97360, saving model to arrTemperature1_best_model_fold1_13_44_52.h5
Epoch 38/40
208/208 [==============================] - 0s 613us/step - loss: 210

Epoch 30/40
208/208 [==============================] - 0s 543us/step - loss: 47.5983 - val_loss: 37.1998

Epoch 00030: val_loss did not improve from 37.01404
Epoch 31/40
208/208 [==============================] - 0s 542us/step - loss: 46.9045 - val_loss: 36.6936

Epoch 00031: val_loss improved from 37.01404 to 36.69357, saving model to arrTemperature1_best_model_fold2_13_45_03.h5
Epoch 32/40
208/208 [==============================] - 0s 525us/step - loss: 45.7476 - val_loss: 36.4430

Epoch 00032: val_loss improved from 36.69357 to 36.44303, saving model to arrTemperature1_best_model_fold2_13_45_03.h5
Epoch 33/40
208/208 [==============================] - 0s 566us/step - loss: 45.2772 - val_loss: 36.1694

Epoch 00033: val_loss improved from 36.44303 to 36.16937, saving model to arrTemperature1_best_model_fold2_13_45_03.h5
Epoch 34/40
208/208 [==============================] - 0s 557us/step - loss: 44.7019 - val_loss: 35.6279

Epoch 00034: val_loss improved from 36.16937 to 35.62794, sav

...

208/208 [==============================] - 0s 503us/step - loss: 61118958.7692 - val_loss: 59579981.8462

Epoch 00028: val_loss improved from 61476566.46154 to 59579981.84615, saving model to arrFans_health3_best_model_fold1_13_52_13.h5
Epoch 29/40
208/208 [==============================] - 0s 499us/step - loss: 59188355.0769 - val_loss: 57622014.1538

Epoch 00029: val_loss improved from 59579981.84615 to 57622014.15385, saving model to arrFans_health3_best_model_fold1_13_52_13.h5
Epoch 30/40
208/208 [==============================] - 0s 500us/step - loss: 57196310.1538 - val_loss: 55622590.1538

Epoch 00030: val_loss improved from 57622014.15385 to 55622590.15385, saving model to arrFans_health3_best_model_fold1_13_52_13.h5
Epoch 31/40
208/208 [==============================] - 0s 498us/step - loss: 55164481.8462 - val_loss: 53570724.9231

Epoch 00031: val_loss improved from 55622590.15385 to 53570724.92308, saving model to arrFans_health3_best_model_fold1_13_52_13.h5
Epoch 32/40
208/

208/208 [==============================] - 0s 493us/step - loss: 69692632.0000 - val_loss: 69000327.3846

Epoch 00021: val_loss improved from 70739656.61538 to 69000327.38462, saving model to arrFans_health3_best_model_fold2_13_52_30.h5
Epoch 22/40
208/208 [==============================] - 0s 496us/step - loss: 67910345.5385 - val_loss: 67164275.0769

Epoch 00022: val_loss improved from 69000327.38462 to 67164275.07692, saving model to arrFans_health3_best_model_fold2_13_52_30.h5
Epoch 23/40
208/208 [==============================] - 0s 496us/step - loss: 66037884.9231 - val_loss: 65238468.9231

Epoch 00023: val_loss improved from 67164275.07692 to 65238468.92308, saving model to arrFans_health3_best_model_fold2_13_52_30.h5
Epoch 24/40
208/208 [==============================] - 0s 488us/step - loss: 64067559.3846 - val_loss: 63222175.3846

Epoch 00024: val_loss improved from 65238468.92308 to 63222175.38462, saving model to arrFans_health3_best_model_fold2_13_52_30.h5
Epoch 25/40
208/

208/208 [==============================] - 0s 504us/step - loss: 77966928.0000 - val_loss: 77659469.5385

Epoch 00014: val_loss improved from 78790579.07692 to 77659469.53846, saving model to arrFans_health3_best_model_fold3_13_52_47.h5
Epoch 15/40
208/208 [==============================] - 0s 513us/step - loss: 76774284.9231 - val_loss: 76454601.8462

Epoch 00015: val_loss improved from 77659469.53846 to 76454601.84615, saving model to arrFans_health3_best_model_fold3_13_52_47.h5
Epoch 16/40
208/208 [==============================] - 0s 494us/step - loss: 75525670.1538 - val_loss: 75181209.8462

Epoch 00016: val_loss improved from 76454601.84615 to 75181209.84615, saving model to arrFans_health3_best_model_fold3_13_52_47.h5
Epoch 17/40
208/208 [==============================] - 0s 495us/step - loss: 74213319.3846 - val_loss: 73805187.3846

Epoch 00017: val_loss improved from 75181209.84615 to 73805187.38462, saving model to arrFans_health3_best_model_fold3_13_52_47.h5
Epoch 18/40
208/

Epoch 7/40
208/208 [==============================] - 0s 553us/step - loss: 961.5859 - val_loss: 1108.0097

Epoch 00007: val_loss did not improve from 1067.05771
Epoch 8/40
208/208 [==============================] - 0s 558us/step - loss: 987.4107 - val_loss: 1057.8566

Epoch 00008: val_loss improved from 1067.05771 to 1057.85660, saving model to arrPower_usage0_best_model_fold1_13_53_28.h5
Epoch 9/40
208/208 [==============================] - 0s 569us/step - loss: 864.1650 - val_loss: 1015.6391

Epoch 00009: val_loss improved from 1057.85660 to 1015.63907, saving model to arrPower_usage0_best_model_fold1_13_53_28.h5
Epoch 10/40
208/208 [==============================] - 0s 565us/step - loss: 733.3393 - val_loss: 969.9900

Epoch 00010: val_loss improved from 1015.63907 to 969.99002, saving model to arrPower_usage0_best_model_fold1_13_53_28.h5
Epoch 11/40
208/208 [==============================] - 0s 544us/step - loss: 692.5746 - val_loss: 964.4984

Epoch 00011: val_loss improved from 96

208/208 [==============================] - 0s 566us/step - loss: 160.3889 - val_loss: 297.2542

Epoch 00014: val_loss did not improve from 287.49216
Epoch 15/40
208/208 [==============================] - 0s 545us/step - loss: 155.0102 - val_loss: 290.5850

Epoch 00015: val_loss did not improve from 287.49216
Epoch 00015: early stopping
104/104 [==============================] - 0s 1ms/step
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 4ms/step - loss: 5716.8937 - val_loss: 6060.0945

Epoch 00001: val_loss improved from inf to 6060.09450, saving model to arrPower_usage0_best_model_fold3_13_54_07.h5
Epoch 2/40
208/208 [==============================] - 0s 600us/step - loss: 5368.0179 - val_loss: 5287.6269

Epoch 00002: val_loss improved from 6060.09450 to 5287.62692, saving model to arrPower_usage0_best_model_fold3_13_54_07.h5
Epoch 3/40
208/208 [==============================] - 0s 582us/step - loss: 4359.8010 - val_loss: 3702.685

In [19]:
print ('start time: ', start_time)
print ('end time:', end_time)

start time:  2019-10-23 18:09:25.736849
end time: 2019-10-23 18:26:28.002689


In [12]:
# Plot and see data distribution

from matplotlib import pyplot as plt
import seaborn as sns

def plotAttrDataOfId(data, compute, features):
    plt.figure(figsize=(30, 20))
    for i, v in enumerate(features):
        plt.subplot(10, 3, i+1)
        cDf = df[df['compute']==compute]
        plt.plot(cDf['timespan'], cDf[v])
        plt.title(v)
        plt.tight_layout()

In [13]:
# for x in np.random.randint(0, len(computes), 3):
#     plotAttrDataOfId(df, computes[x], features)

In [14]:
def plotDataDistribution(data, features):
    plt.figure(figsize=(30, 10))
    for i, v in enumerate(features):
        plt.subplot(3, 10, i+1)
        sns.distplot(list(data[v].values))
        plt.title(v)
    plt.tight_layout()

In [ ]:
# plotDataDistribution(df, features)

In [15]:
from datetime import datetime
print(datetime.now().strftime("%H_%M_%S"))

17_54_25


In [ ]:
sns.distplot(y_train)

In [ ]:
sns.distplot(y_test)

In [ ]:
print (type(X_dfs[0]))